<a href="https://colab.research.google.com/github/joonasrooben/OMML_JAB/blob/main/OMML_projet_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#OMML PROJECT

### Imports

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import math
import scipy as sp

### Data analysis

In [5]:
data = pd.read_csv('DATA.csv')

In [6]:
data.head(5)

,x1,x2,y
0,1.725110,2.665940,0.101883
1,-1.137973,-1.059821,2.172760
2,1.652338,2.958089,0.118859
3,1.866858,-1.740966,-0.289316
4,-1.760604,1.350072,0.104652


In [7]:
len(data)

250

### Split the dataset

We put 0.255 for the test_size instead of 0.25 to be sure to have 186 observations in the training_data

In [8]:
X_train, X_test, y_train, y_test = train_test_split(data[['x1','x2']], data[['y']], test_size=0.255, random_state=1990243)

In [9]:
n = len(X_train)
n

186

In [10]:
X_train

,x1,x2
236,0.890055,-2.955518
134,0.355821,-2.385734
129,1.795969,0.914778
209,0.070684,-2.177204
52,1.674262,0.182846
...,...,...
244,1.557484,-0.167087
158,-1.730929,-0.657224
33,-1.367034,2.838806
35,-1.733677,-1.672311


### Question 1


#### Question 1.1


**Activation function**

We define the acivation function.

In [127]:
def activationFunction(t, sigma = 1):
  return (np.exp(2*sigma*t)-1)/(np.exp(2*sigma*t)+1)

**Weight Initialization for Tanh**

The current standard approach for initialization of the weights of neural network layers and nodes that use the Sigmoid or TanH activation function is called “glorot” or “xavier” initialization

The xavier initialization method is calculated as a random number with a uniform probability distribution (U) between the range -(1/sqrt(n)) and 1/sqrt(n), where n is the number of inputs to the node.

weight = U [-(1/sqrt(n)), 1/sqrt(n)]

Let's define all the hyperparameters here
Let's put 3 neurons in our hidden layer

In [77]:
N = 3
P = 1
sigma = 1

In [55]:
inputs = X_train
inputs = inputs.to_numpy()
inputs.shape

(186, 2)

In [27]:
def createWeightsForOneNeuron():
  initial_weights_one_neuron = []
  for i in range(0,N):
    initial_weights_one_neuron.append(np.random.uniform(low = -(1/math.sqrt(N)), high = 1/math.sqrt(N)))
  return initial_weights_one_neuron

def create_initial_weights():
  initial_weights = []
  for i in range(0,2):
    initial_weights.append(createWeightsForOneNeuron())
  return initial_weights

weightsW = create_initial_weights()



In [56]:
weightsW = weightsW.to_numpy()
weightsW.shape

(2, 3)

In [57]:
mult = np.matmul(inputs, weightsW)

Create vector of bias


In [59]:
bias = np.ones((N,1))

Creation of the linear function

∑wjixi+bj


In [143]:
linearResult = mult + bias.T
linearResult.shape

(186, 3)

Now the step is the get this number into the activation function

In [126]:
acti = activationFunction(linearResult)
acti.shape

(186, 3)

We generate weigths for hidden layers


In [130]:
def createWeightsForHiddenLayer():
  weightHiddenLayer = []
  for i in range(0,N):
    weightHiddenLayer.append(np.random.uniform(low = -(1/math.sqrt(N)), high = 1/math.sqrt(N)))
  return weightHiddenLayer

In [131]:
v = createWeightsForHiddenLayer()
v = np.array(v)

(3,)

In [163]:
f_x = np.matmul(v,acti.T)
f_x.shape


(186,)

Now we have the MLP. We'll computed the regularized training error function E(v,w,b)

In [167]:
y = y_train
y = y.to_numpy()
y = y.reshape((-1))
y.shape

(186,)

In [169]:
firstTermE = (1/2*P)* sum(f_x - y )

First create the regularization term



In [87]:
omega = [v, weightsW, bias]

In [170]:

def computeRegularizationTerm(omega):
    normV = np.linalg.norm(omega[0]**2)
    normW = np.linalg.norm(omega[1]**2)
    normB = np.linalg.norm(omega[2]**2)
    return normV + normW + normB

omega_norm = computeRegularizationTerm(omega)

regularizationTerm = (P/2)*(omega_norm)

In [171]:
E = firstTermE + regularizationTerm